In [1]:
import pandas as pd
import numpy as np

kiumSet = pd.read_csv('E:\k-ium\TrainSet _1차_복사.csv')


In [2]:
kiumSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190 entries, 0 to 6189
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Findings         4814 non-null   object
 1   Conclusion
      6156 non-null   object
 2   AcuteInfarction  6190 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 145.2+ KB
